# Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Data

In [2]:
import shutil
import os

os.makedirs('data', exist_ok=True)

shutil.copyfile('/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/point_cloud.zip' , './data/point_cloud.zip')
shutil.copyfile('/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/traintestlist.zip' , './data/traintestlist.zip')

!unzip data/point_cloud.zip;
!unzip data/traintestlist.zip;

os.remove("./data/point_cloud.zip")
os.remove('./data/traintestlist.zip')

shutil.move('./point_cloud', './data/point_cloud')

Archive:  data/point_cloud.zip
   creating: point_cloud/
  inflating: point_cloud/187558.h5   
  inflating: point_cloud/280599.h5   
  inflating: point_cloud/105452.h5   
  inflating: point_cloud/105668.h5   
  inflating: point_cloud/110123.h5   
  inflating: point_cloud/110210.h5   
  inflating: point_cloud/110488.h5   
  inflating: point_cloud/110498.h5   
  inflating: point_cloud/110544.h5   
  inflating: point_cloud/110787.h5   
  inflating: point_cloud/111150.h5   
  inflating: point_cloud/111350.h5   
  inflating: point_cloud/112237.h5   
  inflating: point_cloud/112324.h5   
  inflating: point_cloud/112532.h5   
  inflating: point_cloud/112558.h5   
  inflating: point_cloud/112736.h5   
  inflating: point_cloud/112804.h5   
  inflating: point_cloud/113202.h5   
  inflating: point_cloud/113351.h5   
  inflating: point_cloud/113382.h5   
  inflating: point_cloud/113389.h5   
  inflating: point_cloud/113850.h5   
  inflating: point_cloud/114732.h5   
  inflating: point_cloud/114843

Error: ignored

# Import Libraries

In [3]:
# importing the libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import math
import time
import glob
import gc

# for creating validation set
from sklearn.model_selection import train_test_split
# for evaluating the model
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef as mcc

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *

%matplotlib inline

import h5py
import csv

# Load Data

In [4]:
def voxel_grid_from_cloud(cloud, out_depth=16, out_height=32, out_width=32):
    in_depth = np.max(cloud[:, 0])
    in_height = np.max(cloud[:, 1])
    in_width = np.max(cloud[:, 2])

    voxel_grid = np.zeros((out_depth, out_height, out_width), dtype=np.uint8)

    if in_depth >= out_depth:
        depth_transform_ratio = out_depth / (in_depth + 1)
        cloud[:, 0] = np.uint32(cloud[:, 0].astype(float) * depth_transform_ratio)
    if in_height >= out_height:
        height_transform_ratio = out_height / (in_height + 1)
        cloud[:, 1] = np.uint32(cloud[:, 1].astype(float) * height_transform_ratio)
    if in_width >= out_width:
        width_transform_ratio = out_width / (in_width + 1)
        cloud[:, 2] = np.uint32(cloud[:, 2].astype(float) * width_transform_ratio)

    voxel_grid[cloud[:, 0], cloud[:, 1], cloud[:, 2]] = 255

    return voxel_grid

def extract_original_name(name, path, suffix):
    name = name.replace(path, "")
    name = name.replace(suffix, "")
    return name


In [5]:
fold_name = "fold_0"    # specify which train-test split to work with

depth, height, width = 32, 64, 64   # dimension for converting point cloud to voxels

path = "./data/point_cloud/"
train_labels_file = "traintestlist/" + fold_name + "_train.csv"
test_labels_file = "traintestlist/" + fold_name + "_test.csv"


# Load all data files, train list and test list

files = [f for f in glob.glob(path + "*.h5", recursive=True)]

with open(train_labels_file, mode='r') as infile:
    reader = csv.reader(infile)
    train_list = {rows[0]: rows[1] for rows in reader}
    infile.close()

with open(test_labels_file, mode='r') as infile:
    reader = csv.reader(infile)
    test_list = {rows[0]: rows[1] for rows in reader}
    infile.close()

train_len = len(train_list) - 1
test_len = len(test_list) - 1


# Create the data

train_x = np.zeros((train_len, 3, depth, height, width), dtype=np.uint8)
train_y = np.zeros(train_len)

test_x = np.zeros((test_len, 3, depth, height, width), dtype=np.uint8)
test_y = np.zeros(test_len)


train_idx = 0
test_idx = 0

for f in tqdm(files):

  hf = h5py.File(f, 'r')
  cloud1 = hf['cloud1'][:]
  cloud2 = hf['cloud2'][:]
  cloud3 = hf['cloud3'][:]
  hf.close()

  X = np.zeros((3, depth, height, width), dtype=np.uint8)
  X[0, :, :, :] = voxel_grid_from_cloud(cloud1, out_depth=depth, out_height=height, out_width=width)
  X[1, :, :, :] = voxel_grid_from_cloud(cloud2, out_depth=depth, out_height=height, out_width=width)
  X[2, :, :, :] = voxel_grid_from_cloud(cloud3, out_depth=depth, out_height=height, out_width=width)
  
  sample_name = extract_original_name(f, path, ".h5") + ".mp4"
  

  if sample_name in train_list.keys(): 
    train_x[train_idx, :, :, :, :] = X
    train_y[train_idx] = int(train_list[sample_name])
    train_idx = train_idx + 1

  if sample_name in test_list.keys(): 
    test_x[test_idx, :, :, :, :] = X
    test_y[test_idx] = int(test_list[sample_name])
    test_idx = test_idx + 1



train_x = torch.from_numpy(train_x).float()
train_y = torch.from_numpy(train_y).long()

test_x = torch.from_numpy(test_x).float()
test_y = torch.from_numpy(test_y).long()


del files, train_list, test_list
gc.collect()

100%|██████████| 2399/2399 [00:04<00:00, 530.32it/s]


4

In [6]:
# print(train_x.shape, test_x.shape)
# print(train_y.shape, test_y.shape)
print(train_idx, test_idx)
type(train_y)

1798 601


torch.Tensor

# Start editing from here

In [7]:
import torch.utils.data
# batch_size, epoch and iteration
batch_size = 64


# Pytorch train and test sets
train = torch.utils.data.TensorDataset(train_x,train_y)
test = torch.utils.data.TensorDataset(test_x,test_y)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False, num_workers=4)

del train_x, train_y
del test_x, test_y

In [8]:
len(test_loader)   #how many test mini batches 

10

In [14]:
num_classes = 2

# Create CNN Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.conv_layer1 = self._conv_layer_set(3, 32)
        self.conv_layer2 = self._conv_layer_set(32, 64)
        self.fc1 = nn.Linear(75264, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, num_classes)
        self.relu = nn.LeakyReLU()
        self.batch1 = nn.BatchNorm1d(2048)
        self.batch2 = nn.BatchNorm1d(512)
        self.batch3 = nn.BatchNorm1d(128)
        self.drop=nn.Dropout(p=0.5)        
        
    def _conv_layer_set(self, in_c, out_c):
        conv_layer = nn.Sequential(
        nn.Conv3d(in_c, out_c, kernel_size=(3, 3, 3), padding=0),
        nn.LeakyReLU(),
        nn.MaxPool3d((2, 2, 2)),
        )
        return conv_layer
    

    def forward(self, x):
        # Set 1
        out = self.conv_layer1(x)
        out = self.drop(out)
        out = self.conv_layer2(out)
        out = out.view(out.size(0), -1)
        #print(out.shape)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.batch1(out)
        out = self.drop(out)
        
        out = self.fc2(out)
        out = self.relu(out)
        out = self.batch2(out)
        out = self.drop(out)

        out = self.fc3(out)
        out = self.relu(out)
        out = self.batch3(out)
        out = self.drop(out)
        out = self.fc4(out)
        
        return out

#Definition of hyperparameters
'''
n_iters = 10000
num_epochs = n_iters / (train_len/ batch_size)
num_epochs = int(num_epochs)
print(num_epochs)
print(train_len)
print(batch_size)
'''
num_epochs = 200
# Create CNN
model = CNNModel()
#model.cuda()
print(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
#device


# Cross Entropy Loss 
error = nn.CrossEntropyLoss().to(device)   #to use model on gpu

# Optimizer
learning_rate = 5e-3
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)  #added L2 regularization

#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

CNNModel(
  (conv_layer1): Sequential(
    (0): Conv3d(3, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (conv_layer2): Sequential(
    (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=75264, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=2, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (batch1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch3): BatchNorm1d(128, eps=1e-0

In [15]:
MCC_SCORE = -1
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools

In [16]:
def conf_mat(cf, epoch, acc, mcc, y_true, train_loss, train_acc, test_loss, test_acc):
  plt.imshow(cf,cmap=plt.cm.RdYlGn,interpolation='nearest')
  plt.colorbar()
  plt.title('Confusion Matrix without Normalization')
  plt.xlabel('Predicted')
  plt.ylabel('Actual')
  tick_marks = np.arange(len(set(y_true))) # length of classes
  class_labels = ['Non Stalled','Stalled']
  tick_marks
  plt.xticks(tick_marks,class_labels)
  plt.yticks(tick_marks,class_labels)
  # plotting text value inside cells
  thresh = cf.max() / 2.
  for i,j in itertools.product(range(cf.shape[0]),range(cf.shape[1])):
      plt.text(j,i,format(cf[i,j],'d'),horizontalalignment='center',color='white' if cf[i,j] >thresh else 'black')
  #plt.show()
  os.makedirs("confusion_matrix", exist_ok=True)
  plt.savefig(f"confusion_matrix/epoch{epoch}_accuracy{round(acc.item(),3)}_mcc_{round(mcc.item(),3)}.png")
  plt.close('all')



  os.makedirs("loss_acc_curve", exist_ok=True)

  ##creating subplot for loss,acc
  fig1, axs = plt.subplots(4,sharex=True,constrained_layout=True)
  axs[0].plot(train_loss, color = "red") 
  axs[0].set_title("Train loss")
  axs[1].plot(train_acc, color = "blue") 
  axs[1].set_title("Train Accuracy")
  axs[2].plot(test_loss, color = "green")
  axs[2].set_title("Test Loss")
  axs[3].plot(test_acc) 
  axs[3].set_title("Test Accuracy")
  #fig1.tight_layout()
  #plt.show()
  fig1.savefig(f"loss_acc_curve/epoch{epoch}_accuracy{round(acc.item(),3)}_mcc_{round(mcc.item(),3)}.png")
  plt.close(fig1)

In [17]:
def validation(epoch,train_loss, train_acc, test_loss, test_acc):
  correct = 0
  total = 0
  t_loss = 0
  y_true = []
  y_pred = []
  # Iterate through test dataset
  model.eval()
  for images, labels in test_loader:
      #print(f"labels size are {labels.shape}")
      y_true = np.append(y_true, labels.numpy())
      images = images.view(-1,3,depth,height,width)
      test = Variable(images.to(device), requires_grad=False)
      labels = Variable(labels.to(device), requires_grad=False)
      with torch.no_grad():
        # Forward propagation
        outputs = model(test)
        t_loss += error(outputs, labels)
        # Get predictions from the maximum value
        predicted = torch.max(outputs.data, 1)[1]
        #print(f"prediction size are {predicted.shape}")
        y_pred = np.append(y_pred, predicted.cpu().numpy())
        # Total number of labels
        total += len(labels)
        correct += (predicted == labels).sum()
  model.train()
  loss = t_loss.cpu().numpy() / float(total)
  test_loss.append(loss)
  accuracy = 100 * correct / float(total)
  test_acc.append(accuracy)

  mcc_score = mcc(y_true, y_pred)
  print(f"MCC score is {mcc_score}")


  global MCC_SCORE
  if MCC_SCORE < mcc_score:
      MCC_SCORE = mcc_score
      os.makedirs("model_checkpoints", exist_ok=True)
      torch.save(model.state_dict(), f"model_checkpoints/3D_pointcloud_SimpleCNN_{epoch}for_cloud_size_32_64_64_acc_{round(accuracy.item(),3)}_mcc_{round(mcc_score.item(),3)}.pth")
  cf =confusion_matrix(y_true, y_pred)
  #print(cf)
  conf_mat(cf, epoch, accuracy, mcc_score, y_true, train_loss, train_acc, test_loss, test_acc)
  #os.makedirs("confusion_matrix", exist_ok=True)
  #plt.savefig(f"confusion_matrix/score{epoch}with_accuracy{final_acc}_mcc_{mcc_score}.png")


  # store loss and iteration
  #loss_list.append(loss.data)
  #test_loss_list.append(t_loss.data)
  #iteration_list.append(count)
  #accuracy_list.append(accuracy)
  # Print Loss
  print('TESTING ---> Epoch: {} Loss: {} Accuracy: {} %'.format(epoch, round(loss,3), round(accuracy.item(),3)))

  '''
  del labels
  del predicted
  del t_loss , outputs
  del images
  del test
  del y_pred, y_true
  del correct
  '''

  return test_loss, test_acc

In [18]:
### TRAINING code
import gc

train_loss = []   #to keep track of loss with respect to number of epoch 
test_loss = []
iteration_list = []
train_acc = []
test_acc = []

for epoch in tqdm(range(num_epochs)):
    count = 1
    accuracy_list = []
    loss_list = []
    for i, (images, labels) in enumerate(train_loader):
        correct = 0
        #print(images.shape)
        images = images.view(-1,3,depth,height,width)
        #train = Variable(images)    #to use on cpu
        #labels = Variable(labels)    #to use on cpu

        train = Variable(images.to(device), requires_grad=True)
        labels = Variable(labels.to(device), requires_grad=False)

        # Clear gradients
        optimizer.zero_grad()
        # Forward propagation
        outputs = model(train)
        # Calculate softmax and ross entropy loss
        loss = error(outputs, labels)
        # Calculating gradients
        loss.backward()
        # Update parameters
        optimizer.step()

        predicted = torch.max(outputs.data, 1)[1]
        correct = (predicted == labels).sum()
        total = len(labels)
        accuracy = 100 * correct / float(total)
        accuracy_list = np.append(accuracy_list, accuracy.cpu().numpy())
        loss_list = np.append(loss_list, loss.detach().cpu().numpy())
        #print(f"Training Mini Batch--> Epoch:{epoch} Iteration:{count} Loss :{loss} Accuracy: {accuracy}")
        count += 1
    final_loss = np.mean(loss_list)
    final_acc = np.mean(accuracy_list)
    print(f"TRAINING ---> Epoch: {epoch} Loss: {round(final_loss,4)} Accuracy: {round(final_acc,4)}")
    train_loss.append(final_loss)
    train_acc.append(final_acc)
    del loss_list, accuracy_list, labels
    del predicted
    del loss , outputs
    del images
    del train
    gc.collect() 
    test_loss , test_acc = validation(epoch, train_loss, train_acc, test_loss, test_acc)
    
            

  0%|          | 0/200 [00:00<?, ?it/s]

TRAINING ---> Epoch: 0 Loss: 0.7175 Accuracy: 61.7636
MCC score is 0.07382230023361153


  0%|          | 1/200 [00:24<1:19:45, 24.05s/it]

TESTING ---> Epoch: 0 Loss: 0.01 Accuracy: 68.885 %
TRAINING ---> Epoch: 1 Loss: 0.6197 Accuracy: 68.6782
MCC score is 0.05448545864179173


  1%|          | 2/200 [00:43<1:14:41, 22.64s/it]

...

TESTING ---> Epoch: 197 Loss: 0.025 Accuracy: 70.549 %
TRAINING ---> Epoch: 198 Loss: 0.13 Accuracy: 96.8391
MCC score is 0.12670569539635088


100%|█████████▉| 199/200 [1:07:58<00:21, 21.05s/it]

TESTING ---> Epoch: 198 Loss: 0.024 Accuracy: 70.549 %
TRAINING ---> Epoch: 199 Loss: 0.0896 Accuracy: 96.9828
MCC score is 0.09877180837279015


100%|██████████| 200/200 [1:08:19<00:00, 20.50s/it]

TESTING ---> Epoch: 199 Loss: 0.02 Accuracy: 69.384 %


In [ ]:
#loss.data
#round(accuracy.data, 3)
#np.mean(accuracy_list)
mcc_score = mcc(y_true, y_pred)
#print(len(y_true))
#print(len(y_pred))
len(accuracy_list)
round(final_acc,4)
print(mcc_score)

In [ ]:

#epoch.append(2)
epoch
train_loss

In [ ]:
import matplotlib.pyplot as plt
# visualization loss 
plt.plot(train_loss)
plt.xlabel("Number of iteration")
plt.ylabel("Loss")
plt.title("CNN: Loss vs Number of iteration")
plt.show()

# visualization accuracy 
plt.plot(train_acc,color = "red")
plt.xlabel("Number of iteration")
plt.ylabel("Accuracy")
plt.title("CNN: Accuracy vs Number of iteration")
plt.show()